<a href="https://colab.research.google.com/github/VOS04/Analise_de_dados_com_Python_Pandas/blob/main/Segundo_exemplo_em_Python_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Segundo exemplo de manipulação de dados usando Python Pandas**
Veremos inserção de dados, remoção de colunas, merge, gráficos

Autor: Valdeno de Oliveira Santos

Data: 9 e 10 de fevereiro de 2023

In [2]:
# Importando biblioteca pandas no Python
import pandas as pd

In [ ]:
combustiveis_df = pd.read_excel("ca-2021-02.xlsx")
display(combustiveis_df.head())

In [ ]:
#Inserção simples de dado
combustiveis_df['Ativo'] = True

display(combustiveis_df.head())

In [ ]:
# Criar uma coluna "Obs" que tenha nela escrito "MELHOR CIDADE" quando a coluna Municipio for igual a SAO PAULO
combustiveis_df['Obs'] = ["MELHOR CIDADE" if municipio == 'SAO PAULO' else None for municipio in combustiveis_df['Municipio']]
display(combustiveis_df.loc[combustiveis_df['Municipio'].isin(['SAO PAULO','INDAIATUBA', 'CAMPINAS', 'SALTO']), ['Municipio', 'Obs']])


In [ ]:
# (por Leandro Rodrigues)
# como preencher uma coluna 'Valor de Venda - Status' que verifica o seguinte:
# se o valor de venda for maior que 6,5 reais, ele fala que tá Caro..caso contrário, está barato
import numpy as np

combustiveis_df['Status do Valor de Venda'] = np.where(combustiveis_df['Valor de Venda'] > 6.5, 'Caro', 'Barato')
display(combustiveis_df[['Revenda', 'Valor de Venda', 'Status do Valor de Venda']])

In [ ]:
# Calcular postos de gasolina por habitante temos na amostragem de 
# combustiveis nov/2021

num_habitantes_df = pd.read_csv("ibge_num_habitantes_estimado.csv", sep=";")
num_habitantes_df.rename(columns={"Estado":"Estado - Sigla"}, inplace=True)
display(num_habitantes_df)


In [ ]:
# Faz um MERGE dos dois dataframes
colunas = ['Municipio', 'Estado - Sigla']
merge_df = combustiveis_df.merge(num_habitantes_df, how="inner", on=colunas)
display(merge_df)
print(merge_df.info())


In [ ]:
#Destruir coluna completamente vazia (todas as linhas são nulas)
merge_df.dropna(axis='columns', inplace=True)
print(merge_df.info())

In [ ]:
colunas=['Regiao - Sigla', 'Nome da Rua', 'Numero Rua', 
         'Bairro', 'Cep', 'Produto', 'Data da Coleta', 'Valor de Venda',
         'Unidade de Medida', 'Bandeira', 'Ativo', 'Status do Valor de Venda']
merge_df.drop(labels=colunas, axis=1, inplace=True)
print(merge_df.info())

In [ ]:
display(merge_df.head(100))

In [ ]:
# Remover a linhas duplicadas
merge_df.drop_duplicates(inplace=True)
display(merge_df.head(100))

In [ ]:
#AQUI O FINAL ESTÁ COM PROBLEMA

#Agrupar e contar quantos postos tem na cidade..
postos_por_municipio_df = merge_df.groupby(by=['Estado - Sigla', 'Municipio', 'NumHabitantes2021']).count()
postos_por_municipio_df.reset_index(inplace=True)
#display(postos_por_municipio_df.info())
postos_por_municipio_df.drop('CNPJ da Revenda', axis=1, inplace=True)
postos_por_municipio_df.rename(columns={"Revenda": "NumPostos"}, inplace=True)

postos_por_municipio_df['PostosPorHabitante'] = postos_por_municipio_df['NumPostos'] / postos_por_municipio_df['NumHabitantes2021']
#display(postos_por_municipio_df.info())
display(postos_por_municipio_df)


In [ ]:
#Exemplo para gerar grafico com a blibioteca matplotlib
import matplotlib.pyplot as plt
plt.hist(combustiveis_df['Valor de Venda'])

# Adicionando titulo no gráfico
plt.title('Valor dos combustiveis - Nov/2021')

#adicionando rotulos ao eixos
plt.xlabel('Preço em Reais')
plt.ylabel('Quantidade de Coletas')

# Traça a linha vermelha tracejada com o preço médio
# linestyle='dashed' tipo de linha que voce vai usar 
# linewidth=1 largura da linha de acordo coom o valor 
plt.axvline(combustiveis_df['Valor de Venda'].mean(), color='red', linestyle='dashed', linewidth=1)



#Plota o grafico no caso do uso em outro hambiente de execusão IDE para mostrar o grafico.
#Obs: Usar o comando sempre no final.
plt.show()

In [ ]:
#Segundo gráfico mostrando a vizualisação do consumo médio
c_mean = combustiveis_df['Valor de Venda'].groupby(by=combustiveis_df['Produto']).mean()
display(c_mean)

In [ ]:
import seaborn as sns

# definindo a área do gráfico
plt.figure(figsize=(7,5))

#Plotar o gráfico
c_mean.plot(
    kind='barh', # tipo de barra, no caso desse exemplo barras horizontais
    xlabel='Tipo de Combustível', # Rotulo do eixo X
    ylabel='Preço reais/litros', # Rotulo do eixo Y
    color='red') # Cor das barras
alpha=0.3

plt.grid() 

#Remover as linhas superior e lateral direita do gráfico 
sns.despine()

plt.show()


In [ ]:
# Cria uma planilha do DataFrame 
display(c_mean)

excel = 'por_litros.xlsx'
c_mean.to_excel(excel, 'Sumário')

In [51]:
# Vamos perssonalizar a planilha Excel...

from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Color, Alignment

# abri o excel no openpyxl
wb = load_workbook(excel) # wb = Workbook

# Pega a planilha certa usando o sheet Name (Nome da planilha).
ws = wb['Sumário'] # Work sheet significa planilha atual, ou planilha ativa, de trabalho.

# Pintar o cabeçalho da tabela de cinza
cinza = PatternFill('solid', fgColor='909090') # função usada para pintar celula
coords = ['A1', 'B1']

for coord in coords:
  ws[coord].fill = cinza

# Onde o preço do combustível for maior ou igual a (6,5) pinta a fonte 
# de vermelho e deixa negrito.

MAX_ROW = ws.max_row

num_linha =2

while (num_linha <= MAX_ROW):
  coord = 'B'+ str(num_linha) # faz a conversão de um valor inteiro em estring
# Condição para verificar e efetuar a mudança na celula
  if ws[coord].value >= 6.5:
    ws[coord].font = Font(bold = True, color='FF0000')

  num_linha = num_linha +1



# Usado para salvar o excel
wb.save(excel)
